In [ ]:
import whisper

# Load the model
model = whisper.load_model("small")

# Transcribe the audio file
result = model.transcribe("..\\Automatic-Illustration-of-Text\\Django\ground_truth_videography_project\\videography_pipeline\\audio\\L0MK7qz13bU.mp3")

print(result)

In [11]:
lrc = f"[ar:someone]\n[ti:title]\n"
for segment in result["segments"]:
    minutes, seconds = divmod(segment["end"], 60)
    lrc += f"[{int(minutes):02d}:{seconds:05.2f}] {segment['text']}\n"

In [ ]:
print(lrc)

In [ ]:
import pytube
from html import unescape
import xml.etree.ElementTree as ElementTree

def xml_to_lrc_captions(xml_captions: str) -> str:
        """Convert xml caption tracks to "SubRip Subtitle (srt)". MODIFIED

        :param str xml_captions:
            XML formatted caption tracks.
        """
        segments = []
        root = ElementTree.fromstring(xml_captions)
        for i, child in enumerate(list(root.findall('body/p'))):
            text = ''.join(child.itertext()).strip()
            if not text:
                continue
            caption = unescape(text.replace("\n", " ").replace("  ", " "),)
            try:
                duration = float(child.attrib["d"])
            except KeyError:
                duration = 0.0
            start = float(child.attrib["t"])
            end = (start + duration)/1000

            minutes, seconds = divmod(end, 60)
            line = f"[{int(minutes):02d}:{seconds:05.02f}]{caption}"
            segments.append(line)
        return "\n".join(segments).strip()

yt = pytube.YouTube("https://www.youtube.com/watch?v=wWsFgl-P3xg", use_oauth=True)
captions = 'en' if 'en' in yt.captions else ('a.en' if 'a.en' in yt.captions else None)
if captions:
    print(f"Grabbing {captions} captions...")
    srt_captions = xml_to_lrc_captions(yt.captions[captions].xml_captions)
    with open(f"{yt.video_id}.srt", 'w', encoding="utf-8") as captions_file:
        captions_file.write(srt_captions)
    